In [208]:
%load_ext line_profiler
%load_ext autoreload
%autoreload 2
from main_dycause_mp_new import *
from main_effcause import effcause_causal_discover

import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from multiprocessing import Pool, RLock, freeze_support

import time
import datetime
import pandas as pd
import os
import numpy as np
import random
import networkx as nx
from sklearn import metrics as skmetrics
import pickle


from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
# rcParams['font.sans-serif'] = ['Arial']
rcParams['pdf.fonttype'] = 42
rcParams['ps.fonttype'] = 42
plt.style.use('default')

freeze_support()  # for Windows support
tqdm.set_lock(RLock())  # for managing output contention

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from utility_funcs.proc_data import load_tcdf_data

datasets, gt_graphs = load_tcdf_data(dir="finance")

In [3]:
datasets[0]

,S0,S1,S2,S3,S4,S5,S6,S7,S8,S9,...,S15,S16,S17,S18,S19,S20,S21,S22,S23,S24
0,0.129999,0.991586,0.490534,1.030064,0.444978,1.208354,0.827543,1.338010,0.759437,0.541619,...,-0.089528,0.507956,0.988139,0.987901,0.310424,-0.029630,0.371462,0.130385,-0.287957,-0.757097
1,0.117729,-0.303972,-1.082923,-0.695533,-0.071504,-0.692810,0.102175,-0.669950,-0.587785,-0.168711,...,-0.532579,-0.594185,-0.396401,-0.735928,-1.771136,-0.548579,-0.357340,-0.914606,-0.480977,-0.567342
2,-0.990932,-1.090502,-0.730285,-0.278989,0.345468,-0.935047,-0.460561,-0.421315,0.100106,-0.831737,...,-0.572841,-0.885619,-0.508891,-0.243055,-0.105957,-0.482223,-0.492560,-0.204166,0.382463,-0.550881
3,1.940631,2.009416,1.683663,1.134586,1.794349,1.398274,-0.106631,1.671531,1.490054,2.175950,...,1.880062,1.461646,1.817153,1.704725,2.065767,1.524978,1.665820,1.425453,2.211056,1.526384
4,0.788248,0.093302,-0.032045,0.589187,0.616736,0.390370,0.965328,0.542184,-0.203809,1.290335,...,-0.238918,0.637504,0.212562,0.933164,0.650985,-0.660012,0.247150,0.771785,0.256871,-0.510581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,0.559481,0.914032,0.613992,0.271723,0.285446,-0.531469,0.232266,-0.161830,-0.049497,0.498747,...,-0.265284,0.477328,0.498301,0.386230,0.125227,0.309916,0.743697,0.417331,-0.077563,0.558236
3996,-0.417690,0.164793,0.077312,0.342314,-0.045468,0.921652,-0.211022,-0.339971,-0.496297,-0.126481,...,-0.859952,-0.550708,-0.918429,-0.519199,-0.806427,-0.294195,-1.218502,-1.511831,-0.723482,-1.860905
3997,-0.170097,0.456529,0.046664,0.206752,-0.498837,0.485043,-0.093383,-0.310373,0.035225,-0.086602,...,-0.183991,-0.077597,-0.149101,-0.421050,-0.365082,0.104056,-0.315758,0.006479,0.686833,0.183096
3998,-0.743060,-1.238800,-1.334410,-0.402677,-0.753884,-0.548000,-0.395417,-0.694155,-0.786199,-1.121678,...,-0.810432,-0.842492,-1.000272,-1.717997,-0.732746,-1.382089,-1.255228,-1.552549,-1.158471,-1.515338


In [38]:
data = datasets[0].to_numpy()[:,:2]

In [173]:
a = []
for test_round in range(10):
    for var_num in range(2, 3):
        data = datasets[0].to_numpy()[:,:var_num]
        verbose=0
        step=500
        lag=5
        local_results_dy, dcc_dy, mat_dy, time_stat_dict_dy = dycause_causal_discover(
            # Data params
            data[:, :],
            # Granger interval based graph construction params
            step=step,
            significant_thres=0.1,
            lag=lag,  # must satisfy: step > 3 * lag + 1
            adaptive_threshold=0.7,
            use_multiprocess=True,
            max_workers=3,
            opt_method="fast_version_3",
            # Debug_params
            verbose=verbose,
            runtime_debug=True,
        )

        local_results_eff, dcc_eff, mat_eff, time_stat_dict_eff = effcause_causal_discover(
            # Data params
            data[:, :],
            # Granger interval based graph construction params
            step=step,
            significant_thres=0.1,
            lag=lag,  # must satisfy: step > 3 * lag + 1
            adaptive_threshold=0.7,
            use_multiprocess=True,
            max_workers=3,
            reuse_invdirection=True,
            rolling_method="pyc",
            # Debug_params
            verbose=verbose,
            runtime_debug=True,
        )

        print("{:4d}\t{:.6f}\t{:.6f}".format(var_num, 
              time_stat_dict_dy['Construct-Impact-Graph-Phase'], 
              time_stat_dict_eff['Construct-Impact-Graph-Phase']))
        a.append((time_stat_dict_dy['Construct-Impact-Graph-Phase'], 
                  time_stat_dict_eff['Construct-Impact-Graph-Phase']))

2021-09-16 02:17:24,153 WARNING: granger_analyze(...) doesn't have enough parameters to name the cache file. Disabling caching.
2021-09-16 02:17:25,808 WARNING: granger_analyze(...) doesn't have enough parameters to name the cache file. Disabling caching.


   2	1.113411	0.527139


2021-09-16 02:17:27,159 WARNING: granger_analyze(...) doesn't have enough parameters to name the cache file. Disabling caching.


   2	0.800734	0.535814


2021-09-16 02:17:28,838 WARNING: granger_analyze(...) doesn't have enough parameters to name the cache file. Disabling caching.


...

   2	0.805574	0.529351


2021-09-16 02:17:36,267 WARNING: granger_analyze(...) doesn't have enough parameters to name the cache file. Disabling caching.


   2	0.935787	0.500805


2021-09-16 02:17:37,661 WARNING: granger_analyze(...) doesn't have enough parameters to name the cache file. Disabling caching.


   2	0.789369	0.589142
   2	0.822122	0.484148


In [174]:
_ = np.array(a)
mean = np.mean(_, axis=0)
std = np.std(_, axis=0)
for i in range(2):
    print("{:.4f}±{:.4f}".format(mean[i], std[i]))
print("{:.4f}".format(mean[0]/mean[1]))

0.8681±0.1335
0.5230±0.0516
1.6601


| Var Num | DyCause (s) | EffCause (s) | Method | Ratio |
| :- | :- | :- | :- | :- | 
| 2 | 2.1124±0.2392 | 1.7446±0.4436 | zyf | 1.2108 |
| 2 | 0.8519±0.1156 | 0.5163±0.0911 | zyf(inv) | 1.6501 |
| 2 | 3.0117±0.5744 | 2.8619±0.5389 | pyc | 1.0524 |
| 2 | 0.8681±0.1335 | 0.5230±0.0516 | pyc(inv) | 1.6601 |

In [45]:
# reuse_invdirection=True
b = np.array(a)
b[:, 0] / b[:, 1]

array([1.80641004, 2.20472284, 2.29374041, 2.40512535, 2.23543593,
       1.85529114, 1.84315557, 1.49414455])

In [47]:
# reuse_invdirection=False
b = np.array(a)
b[:, 0] / b[:, 1]

array([1.52598828, 1.26172153, 1.04947368, 1.99088443, 1.28117348,
       0.82741482, 1.08128527, 1.08730268])

In [ ]:
local_results_dy

In [40]:
local_results_eff, dcc_eff, mat_eff, time_stat_dict_eff = effcause_causal_discover(
    # Data params
    data[:, :],
    # Granger interval based graph construction params
    step=500,
    significant_thres=0.1,
    lag=5,  # must satisfy: step > 3 * lag + 1
    adaptive_threshold=0.7,
    use_multiprocess=True,
    max_workers=3,
    reuse_invdirection=True,
    # Debug_params
    verbose=True,
    runtime_debug=True,
)

print(time_stat_dict_eff['Construct-Impact-Graph-Phase'])

------------------------------------EffCause------------------------------------
--------------------EffCause impact graph construction phase--------------------
          Calculating granger intervals:
               data shape   :(4000, 2)
               lag          :5
               significant  :0.1
               step         :500
               min len      :500
               max len      :4000
               method       :v2
               segment split: [0, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000]
ProcessPoolExecutor starts.

          Saving runtime data to time_stat_dict_20210915_155056.pkl
1.6861674785614014


In [179]:
# local_results_dy['1->0']
import tabulate
res = local_results_dy['0->1']['result_YX']
print(tabulate.tabulate(res, showindex=True, headers=map(str, range(res.shape[1])), floatfmt=".4e"))

              0            1            2            3            4            5            6            7            8
--  -----------  -----------  -----------  -----------  -----------  -----------  -----------  -----------  -----------
 0  -2.0000e+00   4.5963e-05   1.8200e-07   1.2780e-05   5.6550e-08   1.0983e-09   8.0132e-12   1.5195e-10   1.6994e-11
 1  -2.0000e+00  -2.0000e+00   5.2833e-03   6.1762e-03   3.6189e-05   8.0760e-07   7.8919e-09   5.6490e-08   6.3251e-09
 2  -2.0000e+00  -2.0000e+00  -2.0000e+00   9.7676e-03   3.9634e-04   2.3017e-05   3.1478e-07   4.1797e-06   5.0128e-07
 3  -2.0000e+00  -2.0000e+00  -2.0000e+00  -2.0000e+00   6.9257e-03   1.7007e-04   1.5843e-06   1.1390e-05   1.2253e-06
 4  -2.0000e+00  -2.0000e+00  -2.0000e+00  -2.0000e+00  -2.0000e+00   1.1682e-01   8.7523e-04   2.0774e-03   3.4875e-04
 5  -2.0000e+00  -2.0000e+00  -2.0000e+00  -2.0000e+00  -2.0000e+00  -2.0000e+00   5.7881e-03   2.8336e-02   6.3221e-03
 6  -2.0000e+00  -2.0000e+00  -2.0000e+0

In [180]:
# local_results_eff['1->0']
res = local_results_eff['0->1']['result_YX']
print(tabulate.tabulate(res, showindex=True, headers=map(str, range(res.shape[1])), floatfmt=".4e"))

              0            1            2            3            4            5            6            7            8
--  -----------  -----------  -----------  -----------  -----------  -----------  -----------  -----------  -----------
 0  -2.0000e+00   4.5963e-05   1.8200e-07   1.2780e-05   5.6550e-08   1.0983e-09   8.0132e-12   1.5195e-10   1.6994e-11
 1  -2.0000e+00  -2.0000e+00   5.2833e-03   6.1762e-03   3.6189e-05   8.0760e-07   7.8919e-09   5.6490e-08   6.3251e-09
 2  -2.0000e+00  -2.0000e+00  -2.0000e+00   9.7676e-03   3.9634e-04   2.3017e-05   3.1478e-07   4.1797e-06   5.0128e-07
 3  -2.0000e+00  -2.0000e+00  -2.0000e+00  -2.0000e+00   6.9257e-03   1.7007e-04   1.5843e-06   1.1390e-05   1.2253e-06
 4  -2.0000e+00  -2.0000e+00  -2.0000e+00  -2.0000e+00  -2.0000e+00   1.1682e-01   8.7523e-04   2.0774e-03   3.4875e-04
 5  -2.0000e+00  -2.0000e+00  -2.0000e+00  -2.0000e+00  -2.0000e+00  -2.0000e+00   5.7881e-03   2.8336e-02   6.3221e-03
 6  -2.0000e+00  -2.0000e+00  -2.0000e+0

In [176]:
for k in local_results_eff.keys():
    same = (np.all(np.abs(local_results_eff[k]['result_YX'] - local_results_dy[k]['result_YX']) < 1e-10))
    if not same:
        print("Not same within error of 1e-10", k)
        break

KeyError: 'result_YX'

In [53]:
np.abs(local_results_eff[k]['result_YX'] - local_results_dy[k]['result_YX'])

array([[0.00000000e+00, 2.16635995e-05, 6.78155900e-08, 2.64255427e-06,
        1.23793649e-08, 2.33925671e-10, 1.72725528e-12, 2.57595536e-11,
        7.06334695e-24],
       [0.00000000e+00, 0.00000000e+00, 1.60031128e-03, 9.79837991e-04,
        6.92473155e-06, 1.53682930e-07, 1.54879489e-09, 8.55870495e-09,
        1.37063828e-21],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 2.70023147e-03,
        8.84858651e-05, 4.55972138e-06, 6.31405452e-08, 5.84641556e-07,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        2.01822501e-03, 4.11111545e-05, 3.74769564e-07, 1.82733873e-06,
        3.04931861e-19],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 1.78822566e-02, 1.79766340e-04, 2.63800221e-04,
        8.30498864e-17],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.73095090e-03, 3.36974788e-03,
        1.8

In [178]:
for k in local_results_eff.keys():
    ints_eff = sorted(local_results_eff[k]['intervals'])
    ints_dy = sorted([i[1] for i in local_results_dy[k]["intervals"]])
    if not ints_eff==ints_dy:
        print("Not same intervals", k)
        break

In [55]:
ints_eff, ints_dy

([], [(2000, 2500)])

In [122]:
feature=0
target=1
x = datasets[0].to_numpy()[:, [target, feature]]
lag = 5
step=500

from effcause_lib.granger import get_lagged_data, fit_regression_rolling
dta, dtaown, dtajoint = get_lagged_data(x, lag, True, True)

lagged transform: (4000, 2) --> (3995, 11)


In [121]:
from statsmodels.regression.linear_model import OLS
from scipy import stats
end = 195
res2down = OLS(dta[:end, 0], dtaown[:end, :]).fit()
res2djoint = OLS(dta[:end, 0], dtajoint[:end, :]).fit()

fgc1 = (res2down.ssr - res2djoint.ssr) / res2djoint.ssr / lag * res2djoint.df_resid

(fgc1,stats.f.sf(fgc1, lag, res2djoint.df_resid), res2djoint.df_resid,lag,)

(1.7501336726386523, 0.12529846552418225, 184.0, 5)

In [155]:
YX_res2down_all, YX_res2djoint_all = fit_regression_rolling(
    dta, dtaown, dtajoint, lag, step, rolling_method='zyf'
)
nonzero_indices = np.logical_not(np.isnan(YX_res2down_all.ssr)).nonzero()[0]
print(tabulate.tabulate([["YX_res2down"]+list(YX_res2down_all.ssr[nonzero_indices]),
                         ["YX_res2djoint"]+list(YX_res2djoint_all.ssr[nonzero_indices])], 
                         headers=["data"]+list(map(str, nonzero_indices)), 
                        floatfmt=".4e"))

index = 500  - 4000 -1
fgc1 = (
    (YX_res2down_all.ssr[index] - YX_res2djoint_all.ssr[index])
    / YX_res2djoint_all.ssr[index]
    / lag
    * YX_res2djoint_all.df_resid[index]
)
p_value = stats.f.sf(fgc1, lag, YX_res2djoint_all.df_resid[index])

print("sse_part={}\nsse_full={}\nlag={}\ndf_resid_full={}\n"
      "nobs_joint={}\n"
      "nobs_own={}\n"
      "index={}".format(
    YX_res2down_all.ssr[index], YX_res2djoint_all.ssr[index],
    lag, YX_res2djoint_all.df_resid[index], 
    YX_res2djoint_all.nobs[index], 
    YX_res2down_all.nobs[index],
    index
))
print("pvalue=", p_value)

data                   19         494         994        1494        1994        2494        2994        3494        3994
-------------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------
YX_res2down    9.5414e+00  2.0012e+02  4.8487e+02  9.2647e+02  1.2275e+03  1.3939e+03  1.6087e+03  2.2469e+03  2.5012e+03
YX_res2djoint  2.9943e+00  1.8911e+02  4.6578e+02  9.0775e+02  1.2018e+03  1.3659e+03  1.5762e+03  2.2119e+03  2.4643e+03
sse_part=200.1194593276996
sse_full=189.10635472861162
lag=5
df_resid_full=484.0
nobs_joint=495
nobs_own=495
index=-3501
pvalue= 4.596291600133859e-05


In [181]:
YX_res2down_all, YX_res2djoint_all = fit_regression_rolling(
    dta, dtaown, dtajoint, lag, step, rolling_method='pyc'
)
nonzero_indices = np.logical_not(np.isnan(YX_res2down_all.ssr)).nonzero()[0]
print(tabulate.tabulate([["YX_res2down"]+list(YX_res2down_all.ssr[nonzero_indices]),
                         ["YX_res2djoint"]+list(YX_res2djoint_all.ssr[nonzero_indices])], 
                         headers=["data"]+list(map(str, nonzero_indices)), 
                        floatfmt=".4e"))

index = 500  - 4000 -1
fgc1 = (
    (YX_res2down_all.ssr[index] - YX_res2djoint_all.ssr[index])
    / YX_res2djoint_all.ssr[index]
    / lag
    * YX_res2djoint_all.df_resid[index]
)
p_value = stats.f.sf(fgc1, lag, YX_res2djoint_all.df_resid[index])

print("sse_part={}\nsse_full={}\nlag={}\ndf_resid_full={}\n"
      "nobs_joint={}\n"
      "nobs_own={}\n"
      "index={}".format(
    YX_res2down_all.ssr[index], YX_res2djoint_all.ssr[index],
    lag, YX_res2djoint_all.df_resid[index], 
    YX_res2djoint_all.nobs[index], 
    YX_res2down_all.nobs[index],
    index
))
print("pvalue=", p_value)

data                  494         994        1494        1994        2494        2994        3494        3994
-------------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------
YX_res2down    2.0012e+02  4.8487e+02  9.2647e+02  1.2275e+03  1.3939e+03  1.6087e+03  2.2469e+03  2.5012e+03
YX_res2djoint  1.8911e+02  4.6578e+02  9.0775e+02  1.2018e+03  1.3659e+03  1.5762e+03  2.2119e+03  2.4643e+03
sse_part=200.11945932769964
sse_full=189.10635472861162
lag=5
df_resid_full=484.0
nobs_joint=495
nobs_own=495
index=-3501
pvalue= 4.5962916001336215e-05


In [195]:
(YX_res2down_all.ssr[nonzero_indices] - YX_res2djoint_all.ssr[nonzero_indices]) /  YX_res2djoint_all.ssr[nonzero_indices]

array([0.05823762, 0.04097355, 0.02061762, 0.02142373, 0.02052292,
       0.02059963, 0.01581135, 0.01498902])

In [191]:
print(tabulate.tabulate( list((YX_res2down_all.ssr[nonzero_indices] - YX_res2djoint_all.ssr[nonzero_indices]) / YX_res2djoint_all.ssr[nonzero_indices]) ))

TypeError: 'numpy.float64' object is not iterable

In [189]:
stats.f.sf(5, lag, YX_res2djoint_all.df_resid[index])

0.00017780612063471784

In [132]:
len(YX_res2down_all.ssr)

3995

In [131]:
index = 500  - 4000 -1
fgc1 = (
    (YX_res2down_all.ssr[index] - YX_res2djoint_all.ssr[index])
    / YX_res2djoint_all.ssr[index]
    / lag
    * YX_res2djoint_all.df_resid[index]
)
p_value = stats.f.sf(fgc1, lag, YX_res2djoint_all.df_resid[index])

print("sse_part={}\nsse_full={}\nlag={}\ndf_resid_full={}\n"
      "nobs_joint={}\n"
      "nobs_own={}\n"
      "index={}".format(
    YX_res2down_all.ssr[index], YX_res2djoint_all.ssr[index],
    lag, YX_res2djoint_all.df_resid[index], 
    YX_res2djoint_all.nobs[index], 
    YX_res2down_all.nobs[index],
    index
))
print("pvalue=", p_value)


sse_part=200.11945932769964
sse_full=189.10635472861162
lag=5
df_resid_full=484.0
nobs_joint=495
nobs_own=495
index=-3501
pvalue= 4.5962916001336215e-05


In [ ]:
np.logical_not(np.isnan(YX_res2djoint_all.df_resid)).nonzero()

In [ ]:
from effcause_lib.rolling_step import RollingOLS
rolling2down = RollingOLS(
    dta[:, 0], dtaown, window=len(dta), step=step, expanding=True
)
res2down = rolling2down.fit()
print(np.logical_not(np.isnan(res2down.ssr)).nonzero())

In [ ]:
dtaown.shape

In [ ]:
rolling2down._window

In [ ]:
rolling2down._wy

In [ ]:
.fit()
res2djoint_all = RollingOLS(
    dta[:, 0], dtajoint, window=len(dta), step=step, expanding=True
).fit()

In [ ]:
np.ones(10)[:, None]

In [154]:
slice(10,20).start

10